In [1]:
%load_ext autoreload
%autoreload 2

from src.rxn_ctr_mcs import *
from src.utils import load_json, rxn_entry_to_smarts, rm_atom_map_num
from src.pathway_utils import get_reverse_paths_to_starting, create_graph_from_pickaxe
from src.post_processing import *

from minedatabase.pickaxe import Pickaxe
from minedatabase.utils import get_compound_hash

from rdkit.Chem import AllChem

from collections import defaultdict
import pandas as pd
import csv
import pickle

/home/stef/miniconda3/envs/mine/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at /home/stef/miniconda3/envs/mine/lib/python3.7/site-packages/rxnmapper/models/transformers/albert_heads_8_uspto_all_1310k were not used when initializing AlbertModel: ['predictions.LayerNorm.weight', 'predictions.bias', 'predictions.decoder.bias', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expec

In [2]:
# Set params

expansion_dir = '../data/raw_expansions/'
fn = "ccm_v0_to_hopa_gen_4_tan_sample_1_n_samples_1000.pk" # Expansion file name
generations = 4

# Load raw expansion object
pk = Pickaxe()
path = expansion_dir + fn
pk.load_pickled_pickaxe(path)

----------------------------------------
Intializing pickaxe object

Done intializing pickaxe object
----------------------------------------

Loading ../data/raw_expansions/ccm_v0_to_hopa_gen_4_tan_sample_1_n_samples_1000.pk pickled data.
Loaded 69113 compounds
Loaded 91796 reactions
Loaded 3604 operators
Loaded 1 targets
Took 5.1686625480651855


In [3]:
# Create the initial graph

DG, rxn, edge = create_graph_from_pickaxe(pk, "Biology")
starting_nodes = []
bad_nodes = []
for n in DG.nodes():
    try:
        if DG.nodes()[n]["Type"] == "Starting Compound":
            starting_nodes.append(n)
    except:
        bad_nodes.append(n)

RDKit ERROR: [11:35:50] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
[11:35:50] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34

RDKit ERROR: [11:35:55] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
[11:35:55] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34

RDKit ERROR: [11:35:55] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
[11:35:55] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34

RDKit ERROR: [11:35:55] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
[11:35:55] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34

RDKit ERROR: [11:35:55] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
[11:35:55] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34

RDKit ERROR: [11:35:55] Can't kekulize mol.  Unkek

In [4]:
# Get pathways
max_depth = generations * 2
paths = defaultdict(list)

# Specify Targets / Starting Cpds
# target_cids = [get_compound_hash(smi)[0] for smi in pk.target_smiles]
# target_names = [target_smi_name.loc[smi, "id"] for smi in pk.target_smiles]
target_cids, target_names = [], []
for k,v in pk.targets.items():
    target_cids.append(get_compound_hash(v['SMILES'])[0])
    target_names.append(v['ID'])

starting_cpds = [get_compound_hash(val["SMILES"])[0] for val in pk.compounds.values() if val["Type"].startswith("Start")]

# Loop through targets and get pathways from targets to starting compounds
for i, this_target in enumerate(target_cids):
    this_paths = get_reverse_paths_to_starting(DG, begin_node=this_target, end_nodes=starting_cpds, max_depth=max_depth)
    # If we find paths then reverse those paths and assign to a dictionary
    if this_paths:
        this_paths = list(set([tuple(path[1::2]) for path in [[*reversed(ind_path)] for ind_path in this_paths]]))
        for elt in this_paths:
            for r in pk.reactions[elt[0]]["Reactants"]:
                if r[-1] in starting_cpds:
                    s_name = pk.compounds[r[-1]]["ID"]
                    t_name = target_names[i]
                    paths[(s_name, t_name)].append(pathway(rhashes=elt, starter_hash=r[-1], target_hash=this_target)) 

In [5]:
# Make predicted reaction dict

pred_rxns = {}
degen_rhashes = defaultdict(lambda : 1)
for st_pair in paths:
    for elt in paths[st_pair]:
        for this_rhash in elt.rhashes:
            if this_rhash not in pred_rxns:
                rxn_sma = rxn_hash_2_rxn_sma(this_rhash, pk)
                pred_rxns[this_rhash] = reaction(this_rhash, rxn_sma)
            else:
                degen_rhashes[this_rhash] += 1


RDKit ERROR: [11:37:57] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
[11:37:57] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34

RDKit ERROR: [11:37:57] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
[11:37:57] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34

RDKit ERROR: [11:37:57] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
[11:37:57] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34

RDKit ERROR: [11:37:57] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
[11:37:57] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34

RDKit ERROR: [11:37:57] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
RDKit ERROR: [11:37:57] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
[11:37:57] Can't kekuliz

In [6]:
# Load in IMT rule mapping

# Load rules
rules_path = '../data/rules/JN3604IMT_rules.tsv'
rule_df = pd.read_csv(rules_path, delimiter='\t')
rule_df.set_index('Name', inplace=True)

# Load mapping
rxn2rule = {}
db_names = ['_mc_v21', '_brenda', '_kegg']
suffix = '_imt_rules_enforce_cof.csv'
for name in db_names:
    mapping_path = '../data/mapping/mapping' + name + suffix
    with open(mapping_path, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) == 1:
                rxn2rule[row[0]] = []
            else:
                rxn2rule[row[0]] = row[1:]

# Make rule2rxn
rule2rxn = {}
for k,v in rxn2rule.items():
    for elt in v:
        if elt not in rule2rxn:
            rule2rxn[elt] = [k]
        else:
            rule2rxn[elt].append(k)

# Load all known reaction json entries into dict
known_rxns = {}
pref = '../data/mapping/'
suffs = ['mc_v21_as_is.json', 'brenda_as_is.json', 'kegg_as_is.json']
for elt in suffs:
    known_rxns.update(load_json(pref + elt))

In [7]:
# Populate reaction objects in rxn dict w/ known reactions

for k, v in pred_rxns.items():
    this_rules = list(pk.reactions[k]["Operators"])
    this_known_rxns = []
    for elt in this_rules:
        if elt in rule2rxn:
            this_rxn_ids = rule2rxn[elt]
            for this_id in this_rxn_ids:
                this_sma = rxn_entry_to_smarts(known_rxns[this_id])
                this_known_rxns.append((None, this_sma, this_id))
    
    v.known_rxns = [list(elt) for elt in set(this_known_rxns)]


In [8]:
print(len(pred_rxns))
for k,v in paths.items():
    print(k, len(v))

1137
('malate', 'hopa') 2110
('pyruvate', 'hopa') 985
('oxaloacetate', 'hopa') 1624
('succinate', 'hopa') 291
('citrate', 'hopa') 561
('fumarate', 'hopa') 15
('acetate', 'hopa') 6


In [9]:
pr_am_errors = [] # Track predicted rxn am errors
kr_am_errors = [] # Track known rxn am errors
alignment_issues = [] # Track substrate alignment issues

# Populate pred_rxns, known rxn prc-mcs slot
for x in range(len(pred_rxns.keys())):
    h = list(pred_rxns.keys())[x]
    rxn_sma1 = pred_rxns[h].smarts

    # Skip pred reactions that trigger RXNMapper atom mapping errors
    try:
        am_rxn_sma1 = atom_map(rxn_sma1)
    except:
        pr_am_errors.append(h)
        continue

    a = 0 # Number known rxns analyzed
    for z, kr in enumerate(pred_rxns[h].known_rxns):
        rxn_sma2 = kr[1]

        # Catch stoichiometry mismatches stemming from pickaxe, early post-processing
        if tuple([len(elt.split('.')) for elt in rxn_sma2.split('>>')]) != tuple([len(elt.split('.')) for elt in rxn_sma1.split('>>')]):
            print(x, z, 'stoich_error')
            continue

        # Skip pred reactions that trigger RXNMapper atom mapping errors
        try:
            am_rxn_sma2 = atom_map(rxn_sma2)
        except:
            kr_am_errors.append((h, z, kr[-1]))
            continue

        # Construct reaction objects
        rxns = []
        for elt in [am_rxn_sma1, am_rxn_sma2]:
            temp = AllChem.ReactionFromSmarts(elt, useSmiles=True)
            temp.Initialize()
            rxns.append(temp)

        rc_atoms = [elt.GetReactingAtoms() for elt in rxns] # Get reaction center atom idxs

        # Construct rxn ctr mol objs
        try: # REMOVE after addressing KekulizationException in get_sub_mol
            rcs = []
            for i, t_rxn in enumerate(rxns):
                temp = []
                for j, t_mol in enumerate(t_rxn.GetReactants()):
                    temp.append(get_sub_mol(t_mol, rc_atoms[i][j]))
                rcs.append(temp)
        except:
            continue

        # Align substrates of the 2 reactions
        rc_idxs = [] # Each element: (idx for rxn 1, idx for rxn 2)
        remaining = [[i for i in range(len(elt))] for elt in rcs]
        while (len(remaining[0]) > 0) & (len(remaining[1]) > 0):
            idx_pair = align_substrates(rcs, remaining)

            if idx_pair is None:
                break
            else:
                rc_idxs.append(idx_pair)
                remaining[0].remove(idx_pair[0])
                remaining[1].remove(idx_pair[1])

        # Skip if you haven't aligned every reactant pred to known
        if len(rc_idxs) < len(rxn_sma1.split('>>')[0].split('.')):
            alignment_issues.append((h, z, kr[-1]))
            continue

        # For reaction 2 (known reaction) Re-order rcs, rc_atoms,
        # internal order of reactants in the reaction object in rxns
        # and the smarts stored in the known_reactions attribute of the
        # associated predicted reaction

        # Sort reaction 2 rc_idxs by reaction 1 rc_idxs
        rxn_1_rc_idxs, rxn_2_rc_idxs = list(zip(*rc_idxs))
        if rxn_1_rc_idxs != rxn_2_rc_idxs:
            rxn_2_rc_idxs, rxn_1_rc_idxs = sort_x_by_y(rxn_2_rc_idxs, rxn_1_rc_idxs)

            # Re-order atom-mapped smarts string, and then update known_rxns entry
            # with de-atom-mapped version of this string because atom mapper changes
            # reactant order and its this order that rcs, rcatoms, rc_idxs all come from
            am_ro_sma2 = am_rxn_sma2.split('>>')[0].split('.') # Get list of reactant strings
            am_ro_sma2 = '.'.join([am_ro_sma2[elt] for elt in rxn_2_rc_idxs]) # Re-join in new order
            am_rxn_sma2 = am_ro_sma2 + '>>' + am_rxn_sma2.split('>>')[1] # Join with products side

            # Re-construct reaction object from re-ordered, am smarts
            foo = rxns[1]
            temp = AllChem.ReactionFromSmarts(am_rxn_sma2, useSmiles=True)
            temp.Initialize()
            rxns[1] = temp
            bar = rxns[1]

            rc_atoms[1] = rxns[1].GetReactingAtoms() # Update rc_atoms
            rcs[1] = [get_sub_mol(elt, rc_atoms[1][i]) for i, elt in enumerate(rxns[1].GetReactants())] # Update rc mol obj
        
        pred_rxns[h].known_rxns[z][1] = rm_atom_map_num(am_rxn_sma2) # Update known_reaction entry w/ de-am smarts
        rxns = align_atom_map_nums(rxns, rcs, rc_atoms)

        # Compute MCS seeded by reaction center
        prc_mcs = get_prc_mcs(rxns, rcs, rc_atoms) 
        pred_rxns[h].known_rxns[z][0] = prc_mcs # Update pred_rxns
        
        a += 1 # Count known rxn analyzed
        pred_rxns[h].smarts = rm_atom_map_num(am_rxn_sma1) # Update pred_rxn smarts w/ de-am smarts

    print(x, ':', a / (z+1), 'of', z+1)

Token indices sequence length is longer than the specified maximum sequence length for this model (703 > 512). Running this sequence through the model will result in indexing errors


0 : 0.0 of 40
1 : 0.0 of 17
2 : 1.0 of 15
3 : 0.9459459459459459 of 37
4 : 0.0 of 86
5 : 1.0 of 30
6 : 0.975609756097561 of 41
7 : 0.975609756097561 of 41
8 : 1.0 of 6
9 : 0.723404255319149 of 47
10 : 0.15 of 40
11 : 0.20833333333333334 of 24
12 : 0.0 of 83
13 : 0.9459459459459459 of 37
14 : 0.0 of 26
15 : 0.15 of 40
16 : 1.0 of 18
17 : 0.6086956521739131 of 23
18 : 0.0 of 26
19 : 0.08139534883720931 of 86
20 : 0.0 of 6
21 : 0.0 of 26
22 : 1.0 of 28
23 : 0.9090909090909091 of 11
24 : 0.12790697674418605 of 86
25 : 0.23076923076923078 of 26
26 : 0.30666666666666664 of 75
27 : 0.0 of 26
28 : 1.0 of 7
29 : 0.0 of 26
30 : 0.6086956521739131 of 23
31 : 0.961038961038961 of 77
32 : 0.6046511627906976 of 86
33 : 0.0 of 41
34 : 0.8823529411764706 of 17
35 : 0.975609756097561 of 41
36 : 0.0 of 83
37 : 0.15 of 40
38 : 0.13636363636363635 of 88
39 : 0.9285714285714286 of 28
40 : 0.6666666666666666 of 6
41 : 0.0 of 50
42 : 0.8823529411764706 of 17
43 : 0.0 of 83
44 : 0.75 of 8
45 : 0.1363636363636

RDKit ERROR: [11:43:43] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
[11:43:43] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34



128 : 0.0 of 23
129 : 0.15 of 40
130 : 0.20833333333333334 of 24
131 : 1.0 of 10
132 : 0.0 of 41
133 : 0.0 of 26
134 : 0.0 of 40
135 : 0.15 of 40
136 : 0.8823529411764706 of 17
137 : 0.9459459459459459 of 37
138 : 0.0 of 41
139 : 0.975609756097561 of 41
140 : 0.3 of 10
141 : 0.0 of 26
142 : 0.0 of 83
143 : 0.15 of 40
144 : 0.13636363636363635 of 88
145 : 1.0 of 98
146 : 0.13636363636363635 of 88
147 : 0.0 of 83
148 : 1.0 of 21
149 : 0.15 of 40


RDKit ERROR: [11:44:37] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34
RDKit ERROR: 
[11:44:37] Can't kekulize mol.  Unkekulized atoms: 19 23 26 27 28 29 30 31 33 34



...

1052 : 0.953757225433526 of 173
1053 : 0.967741935483871 of 31
1054 : 0.9367088607594937 of 79
1055 : 0.15 of 40


RDKit ERROR: [12:25:21] non-ring atom 2 marked aromatic
[12:25:21] non-ring atom 2 marked aromatic


1056 : 0.012903225806451613 of 465
1057 : 0.0 of 40
1058 : 0.8059701492537313 of 335
1059 : 0.52 of 50
1060 : 0.0 of 88
1061 : 0.13636363636363635 of 88
1062 : 0.5461538461538461 of 130
1063 : 1.0 of 3
1064 0 stoich_error
1064 1 stoich_error
1064 2 stoich_error
1064 3 stoich_error
1064 : 0.0 of 4
1065 : 0.5461538461538461 of 130
1066 : 1.0 of 7
1067 : 0.15 of 40
1068 : 1.0 of 10
1069 : 0.15 of 40
1070 : 0.15 of 40
1071 : 0.2631578947368421 of 19
1072 : 0.6774193548387096 of 31
1074 : 0.08139534883720931 of 86
1075 : 0.723404255319149 of 47
1076 : 0.6086956521739131 of 23
1077 : 0.0 of 24
1078 : 0.12790697674418605 of 86
1079 : 0.6533333333333333 of 75
1080 : 0.15 of 40
1081 : 0.0 of 6
1082 : 0.0 of 88
1083 : 0.0 of 24
1084 : 0.7368421052631579 of 19
1085 : 0.9666666666666667 of 30
1086 : 1.0 of 2


RDKit ERROR: [12:27:00] non-ring atom 3 marked aromatic
[12:27:00] non-ring atom 3 marked aromatic


1087 : 0.0 of 173
1088 : 0.15 of 40
1089 : 1.0 of 18
1090 : 0.15 of 40
1091 : 0.0 of 86
1092 : 0.15 of 40
1093 : 1.0 of 10
1094 : 0.12790697674418605 of 86
1095 : 0.0 of 88
1096 : 1.0 of 7
1097 : 0.15 of 40
1098 : 0.723404255319149 of 47
1099 : 1.0 of 2
1100 : 0.0 of 20
1101 : 0.0 of 83
1102 : 1.0 of 3
1103 : 0.0 of 86
1104 : 0.15 of 40
1105 : 0.0 of 41
1106 : 0.0 of 26
1107 : 0.13636363636363635 of 88
1108 : 0.6533333333333333 of 75
1109 : 0.0 of 6
1110 : 0.7368421052631579 of 19
1111 : 0.723404255319149 of 47
1112 : 0.0 of 88
1113 : 0.723404255319149 of 47
1114 : 0.975609756097561 of 41
1115 : 0.9090909090909091 of 11
1116 : 0.20833333333333334 of 24
1117 : 0.15 of 40
1118 : 0.12790697674418605 of 86
1119 : 0.0 of 24
1120 : 0.20833333333333334 of 24
1121 : 0.0 of 40
1122 : 1.0 of 18
1123 : 0.6086956521739131 of 23
1124 : 0.984 of 125
1125 : 1.0 of 8
1126 : 0.6533333333333333 of 75
1127 : 1.0 of 8
1128 : 1.0 of 6
1129 : 0.0 of 38
1130 : 0.984 of 125
1131 : 0.96875 of 32
1132 : 1.0 of 

In [10]:
# Save reactions dict and paths list (ultimately will replace with expansion object)

rxns_fn = 'predicted_reactions_' + fn
paths_fn = 'paths_' + fn
save_dir = '../data/processed_expansions/'
rxns_path = save_dir + rxns_fn
paths_path = save_dir + paths_fn

with open(rxns_path, 'wb') as f:
    pickle.dump(pred_rxns, f)

with open(paths_path, 'wb') as f:
    pickle.dump(paths, f)

In [11]:
len(alignment_issues), len(pr_am_errors), len(kr_am_errors)

(31366, 38, 4277)